In [ ]:
!pip install Keras-Preprocessing
!apt install unzip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 797.2 kB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras_preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential, Model
from keras import optimizers, losses
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
import pickle
import pandas as pd

In [ ]:
data_dir = "/content/drive/MyDrive/KIMIA_Path_960"

In [ ]:
base_model = InceptionV3(include_top=False, input_shape = (224, 224, 3), weights='imagenet')
# base_model.trainable = True

In [ ]:
# i = 0
for layer in base_model.layers[:132]:
    layer.trainable = False

i = 0;
for layer in base_model.layers:
    # layer.trainable = False
    print(i, layer.name, layer.output)
    i = i+1


In [ ]:
mixed_7a_index = None
for i, layer in enumerate(base_model.layers):
    if layer.name.find('mixed7a'):
        mixed_7a_index = i
        print(i)
        break

# if mixed_7a_index is not None:
#     mixed_7a_layer = base_model.layers[mixed_7a_index]
#     mixed_7a_layers_count = len(mixed_7a_layer.layers)
#     print("Number of sub-layers within Mixed_7a:", mixed_7a_layers_count)
# else:
#     print("Mixed_7a layer not found in the model.")

0


In [ ]:
trdata = ImageDataGenerator()
train = trdata.flow_from_directory(data_dir, target_size=(224,224))

Found 960 images belonging to 20 classes.


In [ ]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(20, activation='softmax'))

opt = optimizers.Adam(learning_rate=0.00001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])


batch_size = 20
epochs = 30
train_generator = train

steps_per_epoch = len(train_generator)


# checkpoint1 = ModelCheckpoint(monitor='accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint1]
history = model.fit(train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch)


Epoch 1/30
30/30 [==============================] - 228s 7s/step - loss: 2.9947 - accuracy: 0.1417
Epoch 2/30
30/30 [==============================] - 6s 183ms/step - loss: 2.0831 - accuracy: 0.3865
Epoch 3/30
30/30 [==============================] - 5s 167ms/step - loss: 1.7401 - accuracy: 0.4583
Epoch 4/30
30/30 [==============================] - 5s 154ms/step - loss: 1.2993 - accuracy: 0.5844
Epoch 5/30
30/30 [==============================] - 6s 187ms/step - loss: 1.1701 - accuracy: 0.6510
Epoch 6/30
30/30 [==============================] - 5s 159ms/step - loss: 0.9495 - accuracy: 0.6990
Epoch 7/30
30/30 [==============================] - 5s 177ms/step - loss: 0.7922 - accuracy: 0.7604
Epoch 8/30
30/30 [==============================] - 5s 158ms/step - loss: 0.6398 - accuracy: 0.8031
Epoch 9/30
30/30 [==============================] - 6s 189ms/step - loss: 0.6063 - accuracy: 0.8281
Epoch 10/30
30/30 [==============================] - 5s 171ms/step - loss: 0.5283 - accuracy: 0.8479


In [ ]:
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
history2 = model.fit(train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch)

Epoch 1/30
30/30 [==============================] - 6s 176ms/step - loss: 0.0698 - accuracy: 0.9802
Epoch 2/30
30/30 [==============================] - 5s 169ms/step - loss: 0.0558 - accuracy: 0.9865
Epoch 3/30
30/30 [==============================] - 6s 185ms/step - loss: 0.0547 - accuracy: 0.9885
Epoch 4/30
30/30 [==============================] - 5s 161ms/step - loss: 0.0561 - accuracy: 0.9885
Epoch 5/30
30/30 [==============================] - 6s 185ms/step - loss: 0.0351 - accuracy: 0.9948
Epoch 6/30
30/30 [==============================] - 5s 178ms/step - loss: 0.0594 - accuracy: 0.9885
Epoch 7/30
30/30 [==============================] - 5s 160ms/step - loss: 0.0525 - accuracy: 0.9854
Epoch 8/30
30/30 [==============================] - 6s 191ms/step - loss: 0.0554 - accuracy: 0.9854
Epoch 9/30
30/30 [==============================] - 5s 160ms/step - loss: 0.0611 - accuracy: 0.9844
Epoch 10/30
30/30 [==============================] - 5s 171ms/step - loss: 0.0330 - accuracy: 0.9948

In [ ]:
# for layers in
model.save('my_model_2.h5')

NameError: name 'model' is not defined

In [ ]:
for layers in model.layers:
  print(layers.name)

inception_v3
flatten
dense
dropout
dense_1


In [ ]:
dense_layer = 'dense'
feature_model = Model(inputs = model.input, outputs = model.get_layer(dense_layer).output)

image_size = 224
features_list = []
filenames_list = []

import os
import cv2
import numpy as np

labels = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T']
for i in labels:
    folderPath = os.path.join('/content/drive/MyDrive/KIMIA_Path_960',i)
    for j in sorted(os.listdir(folderPath)):
        print(j)
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))

        features = feature_model.predict(np.expand_dims(img, axis=0))
        filenames_list.append(i)
        features_list.append(features.flatten())

features_df = pd.DataFrame(features_list)
features_df['Filename'] = filenames_list

features_df.to_excel('inception_V3_features_1.xlsx', index = False)

A1.tif
1/1 [==============================] - 3s 3s/step
A10.tif
1/1 [==============================] - 0s 25ms/step
A11.tif
1/1 [==============================] - 0s 24ms/step
A12.tif
1/1 [==============================] - 0s 24ms/step
A13.tif
1/1 [==============================] - 0s 23ms/step
A14.tif
1/1 [==============================] - 0s 23ms/step
A15.tif
1/1 [==============================] - 0s 26ms/step
A16.tif
1/1 [==============================] - 0s 25ms/step
A17.tif
1/1 [==============================] - 0s 25ms/step
A18.tif
1/1 [==============================] - 0s 25ms/step
A19.tif
1/1 [==============================] - 0s 25ms/step
A2.tif
1/1 [==============================] - 0s 25ms/step
A20.tif
1/1 [==============================] - 0s 24ms/step
A21.tif
1/1 [==============================] - 0s 25ms/step
A22.tif
1/1 [==============================] - 0s 25ms/step
A23.tif
1/1 [==============================] - 0s 25ms/step
A24.tif
1/1 [==============================]

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image


class_labels = list(train.class_indices.keys())
for i in range(len(train.filenames))[:10]:
    # Get the next batch of images
    batch = train.next()
    # print(batch)
    # Extract image and label
    image_data = batch[0][0]
    label_index = int(batch[1][0].argmax())

    # Display the image
    # plt.imshow(image.array_to_img(image_data))
    # plt.title(f"Class: {class_labels[label_index]}")
    # plt.show()